In [0]:
#%pip install great_expectations
#%pip install -U jsonschema

Python interpreter will be restarted.
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.1.0
    Not uninstalling traitlets at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-92a4193b-eba5-4ff1-9dc3-90b23a3d7c3b
    Can't uninstall 'traitlets'. No files were found to uninstall.
  Attempting uninstall: tornado
    Found existing installation: tornado 6.1
    Not uninstalling tornado at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-92a4193b-eba5-4ff1-9dc3-90b23a3d7c3b
    Can't uninstall 'tornado'. No files were found to uninstall.
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 22.2.1
    Not uninstalling pyzmq at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-92a4193b-eba5-4ff1-9dc3-90b23a3d7c3b
    Can't uninstall 'pyzmq'. No files were found to

In [0]:
import datetime
import pytz
import pandas as pd
from ruamel import yaml
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig, FilesystemStoreBackendDefaults
 
# Configurations for filestore location and site location
rootDirectory = "/dbfs/great_expectations/"
siteName = "site"
sitePath = "/dbfs/great_expectations/data_docs/"
 

filesystem = FilesystemStoreBackendDefaults(root_directory=rootDirectory)
dataContext = DataContextConfig(store_backend_defaults=filesystem,
                               data_docs_sites={
                                  siteName: {
                                      "class_name": "SiteBuilder",
                                      "store_backend": {
                                          "class_name": "TupleFilesystemStoreBackend",
                                          "base_directory": sitePath,
                                      },
                                      "site_index_builder": {
                                          "class_name": "DefaultSiteIndexBuilder",
                                      }, 
                                  }
                              },
)
context = BaseDataContext(project_config=dataContext)
 
# Configuration for datasource
my_spark_datasource_config = {
    "name": "databricksDataSource",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "SparkDFExecutionEngine"},
    "data_connectors": {
        "databricksConnector": {
            "module_name": "great_expectations.datasource.data_connector",
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": [
                "runId"
            ],
        }
    },
}
 
# Uncomment line below when needing to test the configuration (not needed for production)
# context.test_yaml_config(yaml.dump(my_spark_datasource_config))
 
# Adds configurations to be used
context.add_datasource(**my_spark_datasource_config)

Out[2]: <great_expectations.datasource.new_datasource.Datasource at 0x7f10c61787f0>

In [0]:
def getBatch(dataframe, testingCategory):
  return RuntimeBatchRequest(
      datasource_name="databricksDataSource",
      data_connector_name="databricksConnector",
      data_asset_name= testingCategory, 
      batch_identifiers={
          "runId": f"{datetime.datetime.now().strftime('%Y%m%d-%H%M%S-%f')}"
      },
      runtime_parameters={"batch_data": dataframe},
  )
  
def createExpecationSuite(expectation_suite):
  context.create_expectation_suite(expectation_suite_name=expectation_suite, overwrite_existing=True)
 
def getTestingSuite(dataframe, testingCategory, expectationSuite):
  createExpecationSuite(expectationSuite)
  batch = getBatch(df, testingCategory)
  return (context.get_validator(
    batch_request= batch,
    expectation_suite_name=expectationSuite
  ) , batch)
  
def runTests(tests, batch):
  runName = f"{datetime.datetime.now(pytz.timezone('America/New_York')).strftime('%Y-%m-%d_%H_%M_%S_EST')}"
  tests.save_expectation_suite(discard_failed_expectations=False)
 
  checkpoint_name = f"{tests.expectation_suite_name}_checkpoint"
  checkpoint_config = {
      "name": checkpoint_name,
      "config_version": 1.0,
      "class_name": "SimpleCheckpoint",
      "run_name_template": runName
  }

  context.add_checkpoint(**checkpoint_config)
  checkpoint_result = context.run_checkpoint(
      checkpoint_name=checkpoint_name,
      validations=[
          {
              "batch_request": batch,
              "expectation_suite_name": tests.expectation_suite_name,
          }
      ],
  )

In [0]:
df = spark.read.table("irs_group_catalog.silver.efile_core_2012")

In [0]:
# These tests are run at the table level, Testing Category can be layer such as bronze, silver, gold

tests, batch = getTestingSuite(dataframe = df, testingCategory = "SilverLayer", expectationSuite = "efile_core_2012")

In [0]:
#list of possible expectations 
#https://legacy.docs.greatexpectations.io/en/latest/reference/glossary_of_expectations.html

tests.expect_column_values_to_be_unique(
    column = "EIN",
    mostly = .99
)

for column in df.columns[0:25]:
    tests.expect_column_values_to_not_be_null(
    column = column,
    mostly= 0.95
    )


# Make sure column values are of certain data type    
tests.expect_column_values_to_be_of_type(
    column = "GROSSRECEIPTS",
    type_ = "DoubleType"
)

tests.expect_column_values_to_be_of_type(
    column = "EIN",
    type_ = "LongType"
)

# Make sure table row count is correct value

tests.expect_table_row_count_to_equal(
    value = 249983
)

# Does data follow valid zipcode format
tests.expect_column_values_to_match_regex(
    column = "ZIP", 
    regex="^\d{5}(?:[-\s]\d{4})?$",
    mostly=.9
)

#validate length of EIN
tests.expect_column_value_lengths_to_be_between(
    column = "EIN",
    min_value=8,
    max_value=9,
    mostly=0.95
) 

#Validate that column values are 
# in value Set
tests.expect_column_values_to_be_in_set(
    column = "TERMINATED",
    value_set=["Y", "N", "NA"],
)

tests.expect_column_values_to_be_in_set(
    column = "ORGDISSOLVED",
    value_set=["TRUE", "FALSE", "NA"]
)

#Compare values in two columns
#Column matching also possible
tests.expect_column_pair_values_A_to_be_greater_than_B(
    column_A="ENDYEAR",
    column_B="STYEAR"
)


Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Out[14]: {
  "result": {
    "element_count": 249983,
    "unexpected_count": 93504,
    "unexpected_percent": 37.404143481756755,
    "partial_unexpected_list": [
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "3/31/2013",
        "4/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "3/31/2013",
        "4/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "5/31/2013",
        "6/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "5/31/2013",
        "6/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "7/31/2013",
        "8/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "6/30/2013",
        "7/1/2012"
      ],
      [
        "6/30/2013",
        "

In [0]:
runTests(tests, batch)

Calculating Metrics:   0%|          | 0/190 [00:00<?, ?it/s]

In [0]:
#Optional: add deloitte logo to tests with this code

def addDeloitteLogo():
    
    from bs4 import BeautifulSoup
    import os
    from pathlib import Path

    htmlPaths = []

    for path in Path('/dbfs/great_expectations/data_docs').rglob('*.html'):

        htmlPaths.append(path)

    for html_doc in htmlPaths:

        HtmlFile = open(html_doc, 'r', encoding='utf-8')

        soup = BeautifulSoup(HtmlFile, 'html.parser')
        img = soup.find_all('img', attrs={"class":"NO-CACHE"})[0]
        img_urls = img['src']
        img_urls = img_urls.replace(img_urls, "https://logos-download.com/wp-content/uploads/2016/10/Deloitte_logo_black-700x130.png")
        img['src'] = img_urls
        with open(html_doc, "w", encoding = 'utf-8') as file:

        # prettify the soup object and convert it into a string  
            file.write(str(soup.prettify()))
            
addDeloitteLogo()

In [0]:
dbutils.fs.ls("s3://great-expectations-packages")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-3398616893997182> in <cell line: 1>()
----> 1 dbutils.fs.ls("s3://great-expectations-packages")

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    360                     exc.__context__ = None
    361                     exc.__cause__ = None
--> 362                     raise exc
    363 
    364             return f_with_exception_handling

ExecutionError: An error occurred while calling o382.ls.
: java.nio.file.AccessDeniedException: s3://great-expectations-packages: getFileStatus on s3://great-expectations-packages: com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied; request: GET https://great-expectations-packages.s3.us-east-1.amazonaws.com  {key=[], key=[false], key=[2], key=[2], key=[/]} Hadoop 3.3.4, aws-sdk-java/1.12.189 Linux/5.4.0-1094-aws OpenJDK_64-Bit_Serv

In [0]:
dbutils.fs.ls("/greatExpectationsPackages/")

Out[12]: [FileInfo(path='dbfs:/greatExpectationsPackages', name='greatExpectationsPackages', size=5294514, modificationTime=1677616689000)]